In [31]:
import onnx
from onnx import helper, TensorProto
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.infer_shapes import InferShapes
model = ModelWrapper("../tinyyolo_infershapes.onnx")
split_node = "Conv_7"
nodes = model.graph.node
passed_nodes=[]
for i,n in enumerate(nodes):
    if n.name != split_node:
        passed_nodes.append(n)
    else:
        passed_nodes.append(n)
        s_node = n        
        break
# print(type(list(nodes)))
start_node = s_node.input[0]
s_node_name = s_node.name
# print(out_node)

def find_input_node(x):
    return 'x' in x.input
upstream_nodes = model.find_upstream(start_node,find_input_node)

In [40]:
up_n_ordered = []
for n in reversed(upstream_nodes):
    up_n_ordered.append(n)
ish = model.get_tensor_shape('x')
osh = model.get_tensor_shape(up_n_ordered[-1].output[0])

In [21]:
inputs = helper.make_tensor_value_info('x',TensorProto.FLOAT,ish)
outputs = helper.make_tensor_value_info(up_n_ordered[-1].output[0],TensorProto.FLOAT,osh)

name: "280"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 512
      }
      dim {
        dim_value: 512
      }
    }
  }
}

In [41]:
new_graph = helper.make_graph(
    up_n_ordered,
    "new_graph",
    [inputs],
    [outputs]
)

split_model = ModelWrapper(helper.make_model(new_graph))
split_model = split_model.transform(InferShapes())
split_model = split_model.transform(InferDataTypes())
model_name = "split_model_{}.onnx".format(split_node)
split_model.save(model_name)